In [1]:
import numpy as np
import pandas as pd 
import geopandas as gpd
import matplotlib.pyplot as plt
import osmnx as ox
import networkx as nx
from descartes import PolygonPatch
from shapely.geometry import Point, Polygon, MultiPolygon, LineString
from glob import glob
from tqdm.notebook import tqdm
import shapely.speedups
from shapely import wkt
shapely.speedups.enable()

/home/bahramkhanbaloch/anaconda3/envs/condaenv/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/tmp/ipykernel_362202/542977013.py:13: DeprecationWarning: This function has no longer any effect, and will be removed in a future release. Starting with Shapely 2.0, equivalent speedups are always available
  shapely.speedups.enable()


In [2]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

def load_city_data_by_name(city_name, target_crs="epsg:3005"):
    city = ox.geocode_to_gdf(city_name)
    city = ox.project_gdf(city)
    geometry = city['geometry'].iloc[0]
    geometry_cut = ox.utils_geo._quadrat_cut_geometry(geometry, quadrat_width=500)
    polylist = [poly for poly in geometry_cut.geoms]
    polyframe = gpd.GeoDataFrame(geometry=polylist)
    polyframe.crs = city.crs
    city_data = ox.project_gdf(polyframe, to_latlong=True)
    # city_data = city_data.to_crs(target_crs)  # Reproject to the target CRS
    # city_data['geometry_center'] = city_data['geometry'].centroid
    return city_data


def add_contains_place_column(city_data, places_data, contains_col_name):
    places_points = [Point(x) for x in places_data['geometry']]
    city_data[contains_col_name] = False

    for index, row in city_data.iterrows():
        grid_polygon = row['geometry']
        for place_point in places_points:
            if grid_polygon.contains(place_point):
                city_data.at[index, contains_col_name] = True
                break  # Exit the inner loop after finding a match

    return city_data


def load_poi_data(path, target_crs="epsg:3005"):
    df = pd.read_csv(path)
    poi_data = gpd.GeoDataFrame(
        df.loc[:, [c for c in df.columns if c != "geometry"]],
        geometry=gpd.GeoSeries.from_wkt(df["geometry"]),
        crs="epsg:3005",  # Assuming the original CRS is EPSG:4326 (geographic CRS)
    )
    poi_data = poi_data.to_crs(target_crs)  # Reproject to the target CRS
    poi_data['geometry_center'] = poi_data['geometry'].centroid
    return poi_data

def add_contains_place_column(city_data, poi_data, contains_col_name):
    places_points = poi_data['geometry'].centroid
    city_data[contains_col_name] = 0  # Initialize the column with zero counts

    for index, row in city_data.iterrows():
        grid_polygon = row['geometry']
        count = 0  # Counter for the number of occurrences

        for place_point in places_points:
            if grid_polygon.contains(place_point):
                count += 1

        city_data.at[index, contains_col_name] = count

    return city_data

In [5]:
path_to_downlaoded_data = "."
cities = glob(f"{path_to_downlaoded_data}/GERMANY_DATA/*")
cities = [city.split("/")[-1] for city in cities]

for city in tqdm(cities):
    print(city)
    if city == "Berlin":
        pass
    else:
        continue

    city_name = city
    city_data = load_city_data_by_name(city)
    poi_data_list = list(glob(f"{path_to_downlaoded_data}/GERMANY_DATA/{city_name}/*.csv"))

    for poi_data_path in tqdm(poi_data_list):
        poi_data = load_poi_data(poi_data_path)
        # Add contains_place column with a custom name
        containment_col_name = poi_data_path.split('/')[-1].split('.')[0].split('-')[-1]
        city_data = add_contains_place_column(city_data, poi_data, containment_col_name)
        # Print the updated city data
        # print(containment_col_name, city_data[containment_col_name].value_counts())
    city_data.to_csv(f"/{path_to_downlaoded_data}/aggregated_data_count/{city_name}_data.csv")
    

  0%|          | 0/9 [00:00<?, ?it/s]

karlsruhe
Mainz
Saarbrucken
Trier
Stuttgart
Munich
Frankfurt
Kaiserslautern
Berlin


  0%|          | 0/20 [00:00<?, ?it/s]

/tmp/ipykernel_1066556/2906246506.py:36: DtypeWarning: Columns (36,43,47,52,60,61,63,66,70,72,74,78,80,85,86,87,88,89,90,92,93,96,97,102,105,106,107,108,110,112,113,114,115,116,117,119,120,121,122,123,125,126,127,129,130,131,132,133,134,135,138,139,140,141,142,143,144,145,148,149,152,153,154,155,156,157,158,160,161,163,164,165,166,167,168,171,172,173,176,177,178,179,180,181,182,185,186,187,188,189,190,191,192,193,195,196,197,199,201,202,203,205,207,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,254,255,256,257,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,278,279,280,281,282,283,284,285,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,303,304,305,306,307,308,309,310,312,313,314,317,318,320,321,322,323,324,325,326,327,328,330,331,332,333) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.re

In [6]:
city_data_path = glob("{path_to_downlaoded_data}/aggregated_data_count/*.csv")

all_city_data = []

for city_path in tqdm(city_data_path):
    city_name = city_path.split('/')[-1].split('_')[0]
    city_data = pd.read_csv(city_path)
    city_data['city'] = city_name
    all_city_data.append(city_data)

all_city_data = pd.concat(all_city_data)
all_city_data.to_csv("{path_to_downlaoded_data}/aggregated_data_count/all_city_data.csv")

all_city_data.head()

  0%|          | 0/9 [00:00<?, ?it/s]

,Unnamed: 0,geometry,parking,edges,EV_stations,parking_space,civic,restaurant,park,school,...,university,cinema,library,commercial,retail,townhall,government,residential,city,Berlin_data_onlycenter_
0,0,"POLYGON ((8.4727605 50.099822499999995, 8.4730...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0,Frankfurt,NaN
1,1,"POLYGON ((8.4775730092433 50.10302720327834, 8...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0,Frankfurt,NaN
2,2,"POLYGON ((8.479750879173663 50.09863320231676,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0,Frankfurt,NaN
3,3,"POLYGON ((8.479688060978736 50.10443297769501,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0,Frankfurt,NaN
4,4,"POLYGON ((8.47965547981383 50.107440331063444,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0,Frankfurt,NaN


In [ ]:
city_data_path = glob("{path_to_downlaoded_data}/aggregated_data_count/*.csv")

all_city_data = []

for city_path in tqdm(city_data_path):
    city_name = city_path.split('/')[-1].split('_')[0]
    city_data = pd.read_csv(city_path)
    city_data['city'] = city_name
    all_city_data.append(city_data)

all_city_data = pd.concat(all_city_data)
all_city_data.to_csv("{path_to_downlaoded_data}/aggregated_data_count/all_city_data.csv")

all_city_data.head()

In [3]:
def add_population_column(city_data, poi_data, contains_col_name):
    places_points = poi_data['geometry'].centroid
    city_data[contains_col_name] = None  # Initialize the column with None values

    for city_index, city_row in city_data.iterrows():
        grid_polygon = city_row['geometry']
        value = None  # Placeholder for the value of poi

        for poi_index, place_point in places_points.iteritems():
            if grid_polygon.contains(place_point):
                poi_value = poi_data.at[poi_index, 'population']
                value = poi_value
                break

        city_data.at[city_index, contains_col_name] = value

    return city_data


In [6]:
city_data_path = glob("{path_to_downlaoded_data}/aggregated_data_count/*.csv")

all_city_data = []

for city_path in tqdm(city_data_path):
    if "all_city_data_with_pop" in city_path:
        continue
    city_name = city_path.split('/')[-1].split('_')[0]
    city_data = pd.read_csv(city_path)
    city_data['city'] = city_name
    city_data['geometry'] = city_data['geometry'].apply(wkt.loads)
    city_data = gpd.GeoDataFrame(city_data, geometry='geometry', crs="epsg:3005")

    #load population data
    #{path_to_downlaoded_data}/population_data/pop_data_v1
    population_data_path = f"{path_to_downlaoded_data}/population_data/pop_data_v1/{city_name}, Germany_pop.csv"
    population_data_path = load_poi_data(population_data_path)
    city_data = add_population_column(city_data, population_data_path, 'population')
    all_city_data.append(city_data)
all_city_data = pd.concat(all_city_data)
all_city_data.to_csv("{path_to_downlaoded_data}/aggregated_data_count/all_city_data_with_pop.csv")

all_city_data.head()

  0%|          | 0/10 [00:00<?, ?it/s]

,Unnamed: 0,geometry,parking,edges,EV_stations,parking_space,civic,restaurant,park,school,...,cinema,library,commercial,retail,townhall,government,residential,city,population,Berlin_data_onlycenter_
0,0,"POLYGON ((8.473 50.100, 8.473 50.100, 8.474 50...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.0,0,Frankfurt,9.014051,NaN
1,1,"POLYGON ((8.478 50.103, 8.479 50.104, 8.479 50...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.0,0,Frankfurt,0.0,NaN
2,2,"POLYGON ((8.480 50.099, 8.480 50.096, 8.477 50...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.0,0,Frankfurt,9.014051,NaN
3,3,"POLYGON ((8.480 50.104, 8.481 50.105, 8.481 50...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.0,0,Frankfurt,9.014051,NaN
4,4,"POLYGON ((8.480 50.107, 8.480 50.111, 8.480 50...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.0,0,Frankfurt,0.0,NaN


In [27]:
all_city_data.sum()

/tmp/ipykernel_353250/941868208.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  all_city_data.sum()


Unnamed: 0                     10900439
parking                           38697
edges                            496615
EV_stations                        2105
parking_space                      9241
civic                               701
restaurant                         9960
park                               4693
school                             2348
node                             225657
Community_centre                   1112
place_of_worship                   2204
university                          270
cinema                              165
library                             396
commercial                         7445
retail                             5203
townhall                            103
government                        410.0
residential                       40204
population                 68600.183628
Berlin_data_onlycenter_          3886.0
dtype: object